In [73]:
import cv2
import glob
import os
import numpy as np
import nd2reader
import matplotlib.pyplot as plt
import nd2


Loading paths

In [74]:
paths = glob.glob(os.path.join("E:/instru_projects/LIveDead_spheroids/22mM/","*.nd2"))
print(paths)

['E:/instru_projects/LIveDead_spheroids/22mM\\240630_Uwell_IPN_22mM_2mgml_132h_spheroidculture_LD_10x.nd2', 'E:/instru_projects/LIveDead_spheroids/22mM\\240630_Uwell_IPN_22mM_2mgml_132h_spheroidculture_LD_20x.nd2', 'E:/instru_projects/LIveDead_spheroids/22mM\\240630_Uwell_IPN_22mM_2mgml_132h_spheroidculture_LD_20x_avg.nd2']


Metadata parser

In [ ]:

def metadata(image):
    meta_dict = {}
    meta_dict["n_fields"] = image.metadata['fields_of_view'].stop 
    meta_dict["n_frames"] = image.metadata['num_frames']
 
    meta_dict["z_levels"] =  float(image.metadata["z_coordinates"][:image.metadata["z_levels"].stop][-1]-image.metadata["z_coordinates"][:image.metadata["z_levels"].stop][0])/float(image.metadata["z_levels"].stop)
    meta_dict["n_z_levels"] = image.metadata['z_levels'].stop
    meta_dict["size_z_step"] = meta_dict["z_level"] /meta_dict["n_levels"]
    meta_dict["channels"] = image.metadata['channels']
    meta_dict["n_channels"] = len(meta_dict["channels"])
    meta_dict["m"] = image.metadata['pixel_microns']
    meta_dict["height"] = image.metadata["height"]
    meta_dict["width"] = image.metadata["width"]
 
    return meta_dict
 
 

Fluorescent images - no z-projection

In [93]:
def channel_max_no_z(images, channels, field):
    mx = np.zeros(len(channels))
    for channel_index, channel_name in enumerate(channels):
            frame = images.get_frame_2D(c=channel_index, z=7, v=field)
            mx[channel_index] = max(mx[channel_index], np.max(frame))
    mx = max(mx[0], mx[2])
    return mx

# Function assignign colors of pixels by channels 
def color_channel(fr,chi, img): #BGR order in cv2
    if chi == 0:  # First channel red in RBF
        img[:, :, 2] = fr
    elif chi == 1:  #
        img[:, :, 0] = 0  # brightfield
    elif chi == 2:  # Third channel (e.g., green)
        img[:, :, 1] = fr  # Set green channel
    return img

def saving (img, field, i_path):
    img_res = cv2.resize(img, (800, 600))
    base_name = os.path.splitext(os.path.basename(i_path))[0]
    parts = base_name.split('_')

    output_subdir = os.path.join('C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2/LD/NO_z_projection/', f"{parts[0]}")
    os.makedirs(output_subdir, exist_ok=True)
    
    if parts[0] == '240630':
        if field < 3:
            name = os.path.join(output_subdir, f"MCF10A_{parts[8:]}_{field}.png")
            cv2.imwrite(name, img_res)
        elif field > 4:
            name = os.path.join(output_subdir, f"DCIS_{parts[8:]}_{field}.png")
            cv2.imwrite(name, img_res)
        else:
            name = os.path.join(output_subdir, f"MCF10AT_{parts[8:]}_{field}.png")
            cv2.imwrite(name, img_res)

#function processing field of view
def process_fof(images, channels, field, i_path, mx_values):
    height, width = images.get_frame_2D(c=1, z=7, v=field).shape
    img = np.zeros((height, width, 3), dtype=np.uint8)

    # Loop through each channel and assign colors
    for channel_index, channel_name in enumerate(channels):
        frame = images.get_frame_2D(c=channel_index, z=7, v=field)
        fr_normalized = (frame / mx_values * 255).astype(np.uint8)
        img = color_channel(fr_normalized, channel_index, img)

        saving(img, field, i_path)      

for count, i_path in enumerate(paths):
    with nd2reader.ND2Reader(i_path) as images:
        fields_of_view = images.metadata['fields_of_view'].stop
#        z_levels = images.metadata['z_levels'].stop
        channels = images.metadata['channels']

        # Calculate maximum intensity across all fields of view and z-levels

        # Use a thread pool to parallelize processing of fields of view
        for field in range(fields_of_view):
            mx_value = channel_max_no_z(images, channels, field)
            process_fof(images,channels, field, i_path, mx_value)
            

Fluorescent images with max z-projection

In [95]:
def z_projection(frames):
    # Stack frames along a new axis (Z-axis) and take the maximum along this axis
    return np.max(np.stack(frames, axis=2), axis=2)

def channel_max(images, channels, z_levels, field):
    mx = np.zeros(len(channels))
    for channel_index, channel_name in enumerate(channels):
        z_slices = []
        for level in range(z_levels):
            frame = images.get_frame_2D(c=channel_index, z=level, v=field)
            z_slices.append(frame)
            z_proj_frame = z_projection(z_slices)
            mx[channel_index] = max(mx[channel_index], np.max(z_proj_frame))
    mx = max(mx[0], mx[2])
    return mx

# Function assignign colors of pixels by channels 
def color_channel(fr,chi, img): #BGR order in cv2
    if chi == 0:  # First channel red in RBF
        img[:, :, 2] = fr
    elif chi == 1:  #
        img[:, :, 0] = 0  # brightfield
    elif chi == 2:  # Third channel (e.g., green)
        img[:, :, 1] = fr  # Set green channel
    return img

def saving (img, field, i_path):
    img_res = cv2.resize(img, (800, 600))
    base_name = os.path.splitext(os.path.basename(i_path))[0]
    parts = base_name.split('_')

    output_subdir = os.path.join('C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2/LD/z_projection/', f"{parts[0]}")
    os.makedirs(output_subdir, exist_ok=True)
    
    if parts[0] == '240630':
        if field < 3:
            name = os.path.join(output_subdir, f"MCF10A_{parts[8:]}_{field}.png")
            cv2.imwrite(name, img_res)
        elif field > 4:
            name = os.path.join(output_subdir, f"DCIS_{parts[8:]}_{field}.png")
            cv2.imwrite(name, img_res)
        else:
            name = os.path.join(output_subdir, f"MCF10AT_{parts[8:]}_{field}.png")
            cv2.imwrite(name, img_res)

#function processing field of view
def process_fof(images, channels, field, i_path, mx_values):
    height, width = images.get_frame_2D(c=1, z=7, v=field).shape
    img = np.zeros((height, width, 3), dtype=np.uint8)

    # Loop through each channel and assign colors
    for channel_index, channel_name in enumerate(channels):
        frame = images.get_frame_2D(c=channel_index, z=7, v=field)
        fr_normalized = (frame / mx_values * 255).astype(np.uint8)
        img = color_channel(fr_normalized, channel_index, img)

        saving(img, field, i_path)      

for count, i_path in enumerate(paths):
    with nd2reader.ND2Reader(i_path) as images:
        fields_of_view = images.metadata['fields_of_view'].stop
        z_levels = images.metadata['z_levels'].stop
        channels = images.metadata['channels']

        # Calculate maximum intensity across all fields of view and z-levels

        # Use a thread pool to parallelize processing of fields of view
        for field in range(fields_of_view):
            mx_value = channel_max(images, channels, z_levels, field)
            process_fof(images,channels, field, i_path, mx_value)
            